**Step 1: Analyzing Python Data and Inferring Schema for Namedtuples**

In [1]:
from collections import namedtuple
from decimal import Decimal
from fractions import Fraction

# Sample namedtuple data
Numbers1 = namedtuple('Numbers1', ['a', 'b', 'c', 'd', 'e'])

data1 = [
    Numbers1(a=1.23, b=Decimal('1.15573'), c=1000000000000, d=2, e=Fraction(22, 7)),
    Numbers1(a=4.56, b=Decimal('1.155727349790921717935726'), c=-22, d=5, e=Fraction(1, 3)),
    Numbers1(a=7.89, b=Decimal('1.0'), c=56, d=9, e=Fraction(5, 1))
]


In [2]:
def infer_schema(data, table_name="my_records"):
    # Mapping of Python types to MySQL data types
    type_mapping = {
        int: "BIGINT",
        float: "REAL",
        Decimal: "DECIMAL(40,25)",
        Fraction: "TEXT",
        str: "TEXT"
    }

    # Analyze the first namedtuple to infer types
    first_row = data[0]
    columns = []

    for field, value in first_row._asdict().items():
        py_type = type(value)
        mysql_type = type_mapping.get(py_type, "TEXT")
        columns.append(f"{field} {mysql_type}")

    # Generate the SQL CREATE TABLE statement
    columns_sql = ",\n    ".join(columns)
    create_table_statement = f"CREATE TABLE {table_name} (\n    {columns_sql}\n);"

    return create_table_statement

print(infer_schema(data1))


CREATE TABLE my_records (
    a REAL,
    b DECIMAL(40,25),
    c BIGINT,
    d BIGINT,
    e TEXT
);


**Explanation:**

Python to MySQL Type Mapping: We establish a dictionary that consists of association between Python types and MySQL types, for instance int is equivalent to BIGINT whereas Decimal is equivalent to DECIMAL.

Dynamic Field Analysis: The first namedtuple contains fields which are dynamically analyzed in order to define the corresponding MySQL data type.

SQL Generation: From the schema we have inferred, we proceed to create a CREATE TABLE statement.

**Step 2: Handling NULLable Columns**

In [3]:
def infer_schema_with_nulls(data, table_name="my_records"):
    type_mapping = {
        int: "BIGINT",
        float: "REAL",
        Decimal: "DECIMAL(40,25)",
        Fraction: "TEXT",
        str: "TEXT"
    }

    columns = []

    # Check for None values in the entire dataset
    for field in data[0]._fields:
        field_types = {type(getattr(row, field)) for row in data}
        if None in field_types:
            mysql_type = "TEXT"
            nullable = "NULL"
        else:
            py_type = type(getattr(data[0], field))
            mysql_type = type_mapping.get(py_type, "TEXT")
            nullable = "NOT NULL"

        columns.append(f"{field} {mysql_type} {nullable}")

    columns_sql = ",\n    ".join(columns)
    create_table_statement = f"CREATE TABLE {table_name} (\n    {columns_sql}\n);"

    return create_table_statement

print(infer_schema_with_nulls(data1))


CREATE TABLE my_records (
    a REAL NOT NULL,
    b DECIMAL(40,25) NOT NULL,
    c BIGINT NOT NULL,
    d BIGINT NOT NULL,
    e TEXT NOT NULL
);


**Explanation:**

NULLable Handling: For each column, record if there are any rows containing None for that column; that column will be marked NULL in MySQL schema.

NOT NULL: If there is no None values in the collection, we set the option for the column as NOT NULL.

**Step 3: Handling Dictionaries, Lists, and Custom Objects**

In [4]:
def infer_schema_from_dicts(data, table_name="my_records"):
    type_mapping = {
        int: "BIGINT",
        float: "REAL",
        Decimal: "DECIMAL(40,25)",
        str: "TEXT"
    }

    columns = []

    first_row = data[0]
    for field, value in first_row.items():
        py_type = type(value)
        mysql_type = type_mapping.get(py_type, "TEXT")
        nullable = "NULL" if value is None else "NOT NULL"
        columns.append(f"{field} {mysql_type} {nullable}")

    columns_sql = ",\n    ".join(columns)
    create_table_statement = f"CREATE TABLE {table_name} (\n    {columns_sql}\n);"

    return create_table_statement

# Example data with dictionaries
data_dicts = [
    {"a": 1, "b": None, "c": 3.14, "d": "text"},
    {"a": 2, "b": Decimal("1.5"), "c": 2.71, "d": "more text"}
]

print(infer_schema_from_dicts(data_dicts))


CREATE TABLE my_records (
    a BIGINT NOT NULL,
    b TEXT NULL,
    c REAL NOT NULL,
    d TEXT NOT NULL
);


**Explanation:**

Dictionaries: Array keys then become field names, and the actual values that you can provide define the MySQL type. Handle None as NULL.

Custom Objects: Access attributes as column using getattr() function and get MySQL types from the values of attributes.

Lists/Tuples: To translate these Si elements, an ontology of analyse fields can be assumed, default column names and map None to null.

All the collections automatically produce MySQL CREATE TABLE statements using Python data types.









## Conclusion

In this project, we were dynamically deducing MySQL schemas from other various Python data types such as namedtuples, dictionaries, and any other collections. We mapped Python types to MySQL types and for the columns which can contain NULL’s, we had to account for None values.

### Key Takeaways:
- **Python to MySQL Type Mapping**: Escalante, they all were successfully mapped to MySQL-compatible types of Python data types including int, float, and Decimal.
- **NULL Handling**: Certain columns which contained `None` were labeled as `NULL` when creating the MySQL schema.
- **Extensibility**: To this solution various features related to dictionaries lists and any other form of collections were added.

### Future Enhancements:
1. **Heterogeneous Data Handling**: realise, work with mixed data types Python collections.
2. **Edge Case Testing**: Expand test bounding still further and limit edges, for example to find broken behaviors where field names of dictionaries are missing or defined erroneously or where the row, column or cell structure of the custom object is erroneous.
